# WEBSCRAPING MOVIE RELEASE DATES IN GERMANY

### IMPORT LIBRARIES

In [39]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pprint import pprint
from lxml import html
from lxml.html import fromstring
import urllib.request
from urllib.request import urlopen
import random
import re
import time
import subprocess
from datetime import datetime
import urllib.request
import unidecode
#import scrapy

### SCRAPING TARGET

In [40]:
#URL KINO-ZEIT.DE CINEMA PROGRAM
url='https://www.kino-zeit.de/filme/filmstarts/aktuell'

### SETTING SCOPE
In Germany theatrical releases are typically on Thursdays.
Setting 'weeks' to 1 will return last Thursdays theatrical releases.
Setting 'weeks' to 2 will return last Thursdays and upcoming Thursday's theatrical releases.

In [41]:
#CHOOSING NUMBER OF UPCOMING WEEKS TO SCRAPE
print('How many upcoming weeks would you like to scrape at kino-zeit.de\n---')
weeks=int(input())

How many upcoming weeks would you like to scrape at kino-zeit.de
---
5


### FUNCTIONS

In [42]:
#FUNCTION TO TURN URL INTO HTML
def make_soup(url):
    html=requests.get(url).content
    return BeautifulSoup(html,'html.parser')

#FUNCTION CREATING THE URL TO FLIP TO NEXT WEEKS PAGE
def next_url(soup):
    url=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
    return url

#FUNCTION FLIPPING THROUGH THE WEEKLY OVERVIEWS
#Picking film titles, start dates and the film's detail page
def how_much_soup(url, weeks):
    df_loop=pd.DataFrame(columns=['start','film','director','actor','link','image','distro','country','trailer'])
    lst_start=[]
    lst_film=[]
    lst_link=[]

    for x in range(weeks): #for the number of weeks we are browsing
        
        soup= make_soup(url)
        lst_film.append([i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})])
        lst_start.append([i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})])
        lst_start=re.findall(r'\d\d.\d\d.\d\d\d\d', str(lst_start))
        lst_link.append([f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})])
        url=next_url(soup) #calling the function for the url we flip to for the following week
    
    lst_film=[i for sublst in lst_film for i in sublst]
    df_loop['film']=lst_film
    df_loop['year']=df_loop['film'].str.findall('\d\d\d\d')
    df_loop['year']=df_loop['year'].apply(listToString)
    #df_loop['start']=lst_start
    lst_start=[datetime.strptime(i,'%d.%m.%Y') for i in lst_start]
    df_loop['start']=lst_start
    lst_link= [i for sublst in lst_link for i in sublst]
    df_loop['link']=lst_link
    return (df_loop)

In [43]:
# FUNCTION TO TURN LIST TO STRING
#NEEDED FOR CLEANING UP ACTORS' COLUMN
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [44]:
#FUNCTION GETTING INFOR FORM FILM DETAIL PAGES
def filmdetails(df, link):
    #CREATING OVERVIEW
    regie_list=[]
    darsteller_list=[]
    list_img_url=[]
    list_verleih=[]
    list_prodland=[]
    
    #BROWSING THE FILMS' INDIVIDUAL PAGES FOR FURTHER DETAILS 
    for i in link:
        print(i)
        sub_page=make_soup(i)
        regie=[i.text.replace('\nRegie','').replace('\n\n','').split(',') for i in sub_page.find_all('div',{'class':'field field--name-field-regie field--type-entity-reference field--label-above'})]
        regie_list.append(regie[0])
        darsteller=[i.text.replace('\nDarsteller','').split('\n') for i in sub_page.find_all('div',{'class':'field field--name-field-hauptdarsteller field--type-entity-reference field--label-above'})]
        darsteller_list.append(darsteller)
        img_url=[i['data-big'].split('?') for i in sub_page.find_all('img',{'id':'headerbildbig'})]
        list_img_url.append(i[0] for i in img_url)
        verleih=[i.text.replace('\nFilmverleih','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-distribution-film field--type-entity-reference field--label-above'})]
        list_verleih.append(verleih)
        prodland=[i.text.replace('\nProduktionsland','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-produktionsland field--type-entity-reference field--label-above'})]
        list_prodland.append(prodland)

    #FILLING THE DATAFRAME WITH THOSE NEW DATA
    df['director']=regie_list
    df['actor']=darsteller_list
    df['image']=list_img_url
    df['distro']=list_verleih
    df['country']=list_prodland
    
    #CLEAN ACTORS COLUMN
    a=[]
    for i in range(len(df.actor)):
        if len(df['actor'].loc[i])==2:
            a.append(list(filter(None,df.actor[i][0])))
        else:
            a.append([''])
        
    df['actor']=a
    df['actor']=df['actor'].apply(listToString)

    #TURN LISTS INTO STRINGS TO CLEAN THEM UP
    for i in range(len(df)):
        
        df['film']=df['film'].str.replace('\(\d\d\d\d\)','')
        df['country'][i]=''.join(df['country'][i]).replace('\n',', ')
        df['director'][i]=', '.join(df['director'][i]).replace('\n',', ')
        df['image'][i]=''.join(df['image'][i])
        df['distro'][i]=''.join(df['distro'][i]).replace('\n',', ')
    return df

In [45]:
### DEALING WITH UMLAUT CHARACTERS
### CURRENTLY NOT IN USE
"""def remove_odd_characters(string):
    
    u = 'ü'.encode()
    U = 'Ü'.encode()
    a = 'ä'.encode()
    A = 'Ä'.encode()
    o = 'ö'.encode()
    O = 'Ö'.encode()
    ss = 'ß'.encode()
    e = 'é' .encode()
    e = 'è' .encode()

    string = string.encode()
    string = string.replace(u, b'ue')
    string = string.replace(U, b'Ue')
    string = string.replace(a, b'ae')
    string = string.replace(A, b'Ae')
    string = string.replace(o, b'oe')
    string = string.replace(O, b'Oe')
    string = string.replace(ss, b'ss')
    string = string.replace(e, b'e')

    string = string.decode('utf-8')
    return string"""

"def remove_odd_characters(string):\n    \n    u = 'ü'.encode()\n    U = 'Ü'.encode()\n    a = 'ä'.encode()\n    A = 'Ä'.encode()\n    o = 'ö'.encode()\n    O = 'Ö'.encode()\n    ss = 'ß'.encode()\n    e = 'é' .encode()\n    e = 'è' .encode()\n\n    string = string.encode()\n    string = string.replace(u, b'ue')\n    string = string.replace(U, b'Ue')\n    string = string.replace(a, b'ae')\n    string = string.replace(A, b'Ae')\n    string = string.replace(o, b'oe')\n    string = string.replace(O, b'Oe')\n    string = string.replace(ss, b'ss')\n    string = string.replace(e, b'e')\n\n    string = string.decode('utf-8')\n    return string"

In [46]:
### DEALING WITH FOREIGN LANGUAGE CHARACTERS

def recoding_string (df):
    trailer_list=[]
    for i in range(len(df)):
        search= f"{df['film'][i].replace(' ','_')}_trailer_{df['distro'][i].replace(' ','_')}"
        #search=remove_odd_characters(search) --- calls function remove umlaut
        search=unidecode.unidecode(search) # dealing with foreign language letters so we can include them in youtube search URL
        print(search)
        html = urllib.request.urlopen(f"https://www.youtube.com/results?search_query={search}")
        video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
        if video_ids == []:
            trailer_list.append('no trailer found')
        else:
            youtube_url= f'https://youtu.be/{video_ids[0]}'
            trailer_list.append(youtube_url)
    return trailer_list

# START PROGRAM

In [47]:
#CALL FUNCTION FILM OVERVIEW
df=how_much_soup(url, weeks)
df


,start,film,director,actor,link,image,distro,country,trailer,year
0,2021-11-04,1982 Neunzehnhundertzweiundachtzig (2019),NaN,NaN,https://www.kino-zeit.de/node/52865,NaN,NaN,NaN,NaN,1982 2019
1,2021-11-04,A Dark Song (2016),NaN,NaN,https://www.kino-zeit.de/node/52778,NaN,NaN,NaN,NaN,2016
2,2021-11-04,Ammonite (2020),NaN,NaN,https://www.kino-zeit.de/node/49449,NaN,NaN,NaN,NaN,2020
3,2021-11-04,Bergman Island (2021),NaN,NaN,https://www.kino-zeit.de/node/44759,NaN,NaN,NaN,NaN,2021
4,2021-11-04,Bloom up - Hautnah (2021),NaN,NaN,https://www.kino-zeit.de/node/53367,NaN,NaN,NaN,NaN,2021
...,...,...,...,...,...,...,...,...,...,...
86,2021-12-02,Gunpowder Milkshake (2019),NaN,NaN,https://www.kino-zeit.de/node/51136,NaN,NaN,NaN,NaN,2019
87,2021-12-02,House of Gucci (2021),NaN,NaN,https://www.kino-zeit.de/node/51978,NaN,NaN,NaN,NaN,2021
88,2021-12-02,Plötzlich aufs Land - Eine Tierärztin im Burgu...,NaN,NaN,https://www.kino-zeit.de/node/52261,NaN,NaN,NaN,NaN,2019
89,2021-12-02,Vater - Otac (2020),NaN,NaN,https://www.kino-zeit.de/node/50368,NaN,NaN,NaN,NaN,2020


In [48]:
# Spezical Fall Film dem zB der Regie Credit fehlt erzeugt list our of range fehler. 
#Die entsprechenden Filme muss man entfernen.
df= df[df['film']!='SimsalaGrimm (2020)'].reset_index(drop=True)


In [49]:
#CALL FUNCTION FILM DETAILS
df=filmdetails(df, df['link'])
df

https://www.kino-zeit.de/node/52865
https://www.kino-zeit.de/node/52778
https://www.kino-zeit.de/node/49449
https://www.kino-zeit.de/node/44759
https://www.kino-zeit.de/node/53367
https://www.kino-zeit.de/node/52102
https://www.kino-zeit.de/node/52546
https://www.kino-zeit.de/node/52399
https://www.kino-zeit.de/node/45823
https://www.kino-zeit.de/node/53132
https://www.kino-zeit.de/node/49038
https://www.kino-zeit.de/node/53016
https://www.kino-zeit.de/node/52954
https://www.kino-zeit.de/node/53258
https://www.kino-zeit.de/node/44509
https://www.kino-zeit.de/node/42712
https://www.kino-zeit.de/node/52867
https://www.kino-zeit.de/node/53381
https://www.kino-zeit.de/node/50410
https://www.kino-zeit.de/node/52777
https://www.kino-zeit.de/node/52283
https://www.kino-zeit.de/node/53327
https://www.kino-zeit.de/node/50707
https://www.kino-zeit.de/node/48853
https://www.kino-zeit.de/node/52028
https://www.kino-zeit.de/node/53145
https://www.kino-zeit.de/node/50969
https://www.kino-zeit.de/nod

<ipython-input-44-a815aaeac6f9>:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df['film']=df['film'].str.replace('\(\d\d\d\d\)','')
<ipython-input-44-a815aaeac6f9>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['country'][i]=''.join(df['country'][i]).replace('\n',', ')
<ipython-input-44-a815aaeac6f9>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['director'][i]=', '.join(df['director'][i]).replace('\n',', ')
<ipython-input-44-a815aaeac6f9>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

,start,film,director,actor,link,image,distro,country,trailer,year
0,2021-11-04,1982 Neunzehnhundertzweiundachtzig,Oualid Mouaness,Nadine Labaki Mohamad Dalli Gia Madi Rodrigue ...,https://www.kino-zeit.de/node/52865,https://www.kino-zeit.de/sites/default/files/s...,mec film,"Libanon, USA, Katar, Norwegen",NaN,1982 2019
1,2021-11-04,A Dark Song,Liam Gavin,Steve Oram Catherine Walker Susan Loughnane Ma...,https://www.kino-zeit.de/node/52778,https://www.kino-zeit.de/sites/default/files/s...,Drop-Out Cinema eG,"Irland, Großbritannien",NaN,2016
2,2021-11-04,Ammonite,Francis Lee,Saoirse Ronan Kate Winslet Fiona Shaw Gemma Jo...,https://www.kino-zeit.de/node/49449,https://www.kino-zeit.de/sites/default/files/s...,Tobis Film GmbH,Großbritannien,NaN,2020
3,2021-11-04,Bergman Island,Mia Hansen-Løve,Mia Wasikowska Vicky Krieps Anders Danielsen L...,https://www.kino-zeit.de/node/44759,https://www.kino-zeit.de/sites/default/files/s...,Weltkino Filmverleih,Frankreich,NaN,2021
4,2021-11-04,Bloom up - Hautnah,Mauro Russo Rouge,,https://www.kino-zeit.de/node/53367,https://www.kino-zeit.de/sites/default/files/s...,"Nameless Media, 24 Bilder",Italien,NaN,2021
...,...,...,...,...,...,...,...,...,...,...
85,2021-12-02,Gunpowder Milkshake,Navot Papushado,Karen Gillan Lena Headey Michelle Yeoh Angela ...,https://www.kino-zeit.de/node/51136,https://www.kino-zeit.de/sites/default/files/s...,STUDIOCANAL GmbH Filmverleih,,NaN,2019
86,2021-12-02,House of Gucci,Ridley Scott,Lady Gaga Adam Driver Jared Leto Al Pacino Jer...,https://www.kino-zeit.de/node/51978,https://www.kino-zeit.de/sites/default/files/s...,Universal Pictures International Germany,USA,NaN,2021
87,2021-12-02,Plötzlich aufs Land - Eine Tierärztin im Burgund,Julie Manoukian,Clovis Cornillac Noémie Schmidt Carole Franck,https://www.kino-zeit.de/node/52261,https://www.kino-zeit.de/sites/default/files/s...,"Happy Entertainment, 24 Bilder",Frankreich,NaN,2019
88,2021-12-02,Vater - Otac,Srdan Golubovic,Goran Bogdan Boris Isakovic Nada Sargin Milica...,https://www.kino-zeit.de/node/50368,https://www.kino-zeit.de/sites/default/files/s...,barnsteiner Film,"Serbien, Kroatien, Deutschland",NaN,2020


In [50]:
df['trailer']=recoding_string(df)

1982_Neunzehnhundertzweiundachtzig__trailer_mec_film
A_Dark_Song__trailer_Drop-Out_Cinema_eG
Ammonite__trailer_Tobis_Film_GmbH_
Bergman_Island__trailer_Weltkino_Filmverleih
Bloom_up_-_Hautnah__trailer_Nameless_Media,_24_Bilder
Chasing_Paper_Birds__trailer_deja-vu_film_UG_
Die_Geschichte_meiner_Frau__trailer_Alamode_Filmverleih
Happy_Family_2__trailer_Warner_Bros._Pictures_Germany
Kabul_Kinderheim__trailer_Steppenwolf
Kinder_der_Hoffnung__trailer_Film_Kino_Text
King_Otto__trailer_Filmwelt_Verleihagentur
Kosmetik_des_Bosen__trailer_Koch_Films_GmbH,_Central_Film
Meine_Wunderkammern__trailer_eksystent_distribution_filmverleih
PARIS_kein_Tag_ohne_dich__trailer_jip_Film_&_Verleih
Red_Notice__trailer_Netflix,_Filmwelt_Verleihagentur
Remi_-_Sein_grosstes_Abenteuer__trailer_Der_Filmverleih_GmbH
Schocken_-_Ein_deutsches_Leben__trailer_Salzgeber
The_Doors:_Live_At_The_Bowl_'68_Special_Edition__trailer_Piece_of_Magic_Entertainment
The_Many_Saints_of_Newark__trailer_Warner_Bros._Entertainment_GmbH_

In [51]:
df

,start,film,director,actor,link,image,distro,country,trailer,year
0,2021-11-04,1982 Neunzehnhundertzweiundachtzig,Oualid Mouaness,Nadine Labaki Mohamad Dalli Gia Madi Rodrigue ...,https://www.kino-zeit.de/node/52865,https://www.kino-zeit.de/sites/default/files/s...,mec film,"Libanon, USA, Katar, Norwegen",https://youtu.be/QLqHXtgNmJ4,1982 2019
1,2021-11-04,A Dark Song,Liam Gavin,Steve Oram Catherine Walker Susan Loughnane Ma...,https://www.kino-zeit.de/node/52778,https://www.kino-zeit.de/sites/default/files/s...,Drop-Out Cinema eG,"Irland, Großbritannien",https://youtu.be/qa6Dl0mKbS8,2016
2,2021-11-04,Ammonite,Francis Lee,Saoirse Ronan Kate Winslet Fiona Shaw Gemma Jo...,https://www.kino-zeit.de/node/49449,https://www.kino-zeit.de/sites/default/files/s...,Tobis Film GmbH,Großbritannien,https://youtu.be/yxRckKH1s8I,2020
3,2021-11-04,Bergman Island,Mia Hansen-Løve,Mia Wasikowska Vicky Krieps Anders Danielsen L...,https://www.kino-zeit.de/node/44759,https://www.kino-zeit.de/sites/default/files/s...,Weltkino Filmverleih,Frankreich,https://youtu.be/X3k9vf6PR9o,2021
4,2021-11-04,Bloom up - Hautnah,Mauro Russo Rouge,,https://www.kino-zeit.de/node/53367,https://www.kino-zeit.de/sites/default/files/s...,"Nameless Media, 24 Bilder",Italien,https://youtu.be/Scznf8tOwrY,2021
...,...,...,...,...,...,...,...,...,...,...
85,2021-12-02,Gunpowder Milkshake,Navot Papushado,Karen Gillan Lena Headey Michelle Yeoh Angela ...,https://www.kino-zeit.de/node/51136,https://www.kino-zeit.de/sites/default/files/s...,STUDIOCANAL GmbH Filmverleih,,https://youtu.be/yxuAroBqt2c,2019
86,2021-12-02,House of Gucci,Ridley Scott,Lady Gaga Adam Driver Jared Leto Al Pacino Jer...,https://www.kino-zeit.de/node/51978,https://www.kino-zeit.de/sites/default/files/s...,Universal Pictures International Germany,USA,https://youtu.be/pGi3Bgn7U5U,2021
87,2021-12-02,Plötzlich aufs Land - Eine Tierärztin im Burgund,Julie Manoukian,Clovis Cornillac Noémie Schmidt Carole Franck,https://www.kino-zeit.de/node/52261,https://www.kino-zeit.de/sites/default/files/s...,"Happy Entertainment, 24 Bilder",Frankreich,https://youtu.be/vVHrYoqRWVc,2019
88,2021-12-02,Vater - Otac,Srdan Golubovic,Goran Bogdan Boris Isakovic Nada Sargin Milica...,https://www.kino-zeit.de/node/50368,https://www.kino-zeit.de/sites/default/files/s...,barnsteiner Film,"Serbien, Kroatien, Deutschland",https://youtu.be/Gi8QtN521LE,2020


### Subsetting 
films with production country 'Deutschland' and after a certain date

In [52]:
df=df[(df['country'].str.contains('Deutschland'))&(df['start']>'2021-11-21')]


## EXPORT EXCEL + CSV  WITH TIMESTAMP

In [53]:
timestr = time.strftime("%Y%m%d")
df.to_csv(f'exported_files/Filmstarts_{timestr}.csv')
df.to_excel(f'exported_files/Filmstarts_{timestr}.xlsx')

### Option download images

In [54]:
#def img_download(filmtitle, url):
for i in range(len(df)):
    response = requests.get(df.iloc[i][5])
    file = open(f'exported_files/{df.iloc[i][1]}.jpg', 'wb')
    file.write(response.content)
    file.close()

### Option to have resulting Excel sent by email

In [55]:
import os
import smtplib
from email.message import EmailMessage

email_address=os.environ.get('email_user')
email_password=os.environ.get('email_pass')
smtp_ssl_host=os.environ.get('DFA_mailserver')
smtp_ssl_port = 465

msg= EmailMessage()
msg['Subject']= 'Filmstarts sind exportiert! - dein bot'
msg['From']=email_address
msg['To']='spitzer@deutsche-filmakademie.de'
msg.set_content('Hallo, hallo, \n\nHier ein Mailtest aus Jupyter Notebook.\n\nDas Excel mit den aktuellen Filmstarts ist anbei. Gefiltert nach Filmen mit Produktionsland Deutschland. Und Filmstart nach dem 5.November. Die Fotos sind in der Mediathek\n\nschöne Grüße dein Python Bot!')

with open(f'exported_files/Filmstarts_{timestr}.xlsx', 'rb') as attachment:
    file=attachment.read()

msg.add_attachment(file,maintype='Excel',subtype='xlsx', filename=f'Filmstarts_{timestr}.xlsx')

with smtplib.SMTP_SSL(smtp_ssl_host, smtp_ssl_port) as server:
    server.login(email_address, email_password)
    server.send_message(msg)
    server.quit()

### RINGS BELL WHEN SCRIPT FINISHED

In [56]:
#RINGS THE BELL WHEN SCRIPT IS DONE
audio_file = "service-bell.mp3"
return_code = subprocess.call(["afplay", audio_file])

In [57]:
"""
a=[]
for i in range(len(df.actor)):
        if len(df['actor'].loc[i])==2:
            a.append(list(filter(None,df.actor[i][0])))
        else:
           a.append([''])
df['actor_test']=a
#df['actor_test']=listToString(list(df['actor_test']))
df['actor_test']=df['actor_test'].apply(listToString)
df
"""

"\na=[]\nfor i in range(len(df.actor)):\n        if len(df['actor'].loc[i])==2:\n            a.append(list(filter(None,df.actor[i][0])))\n        else:\n           a.append([''])\ndf['actor_test']=a\n#df['actor_test']=listToString(list(df['actor_test']))\ndf['actor_test']=df['actor_test'].apply(listToString)\ndf\n"